In [ ]:
%pip install numpy pandas matplotlib scikit-learn h5py

In [2]:
import h5py
import scipy.io as sio
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load File

In [3]:
matFilename = '/Users/guancong/Desktop/sch/3110_ITP/BatteryLifeTimeML/Data/2017-05-12_batchdata_updated_struct_errorcorrect.mat'
f = h5py.File(matFilename)

In [6]:
batch = f['batch']
list(batch.keys())

<HDF5 group "/batch" (8 members)>

In [22]:
summary = batch['summary']
cycles = batch['cycles']
np.array(cycles[1,1:5])

array([], dtype=object)

In [ ]:
#remove batteries continuing from batch 1
batch[np.array[[np.arange[8,10+1],np.arange[16,17+1]]]] = []
batch2 = batch
numBat2 = batch2.shape[2-1]
batch_combined = np.array([batch1,batch2])
numBat = numBat1 + numBat2
#remove the batteries that do not finish in Batch 1
batch_combined[np.array[[9,11,13,14,23]]] = []
numBat = numBat - 5
numBat1 = numBat1 - 5

## Outputs

In [ ]:
bat_label = np.zeros((numBat,1))
for i in np.arange(1,numBat1+1).reshape(-1):
    bat_label[i] = batch_combined(i).cycles.shape[2-1]

for i in np.arange(1,numBat2+1).reshape(-1):
    if batch2(i).summary.QDischarge(end()) < 0.88:
        bat_label[i + numBat1] = find(batch2(i).summary.QDischarge < 0.88,1)
    else:
        bat_label[i + numBat1] = batch2(i).cycles.shape[2-1]

bat_label[np.arange[1,5+1]] = np.array([[1852],[2160],[2237],[1434],[1709]])
plt.figure()
hold('on')
for i in np.arange(1,numBat+1).reshape(-1):
    plt.plot(batch_combined(i).summary.QDischarge,'.-')

max_Q = log10(np.amax(bat_label) + 1)
min_Q = log10(np.amin(bat_label) - 1)
bat_label = log10(bat_label)
colormap('jet')
CM = colormap('jet')

## Features

In [ ]:
hump = np.zeros((numBat,1))
for i in np.arange(1,numBat+1).reshape(-1):
    val,hump[i] = np.amax(batch_combined(i).summary.QDischarge(np.arange(1,100+1)))

hump[15] = 19
feat = np.zeros((numBat,1))
plt.figure()

for i in np.arange(1,numBat+1).reshape(-1):
    #Q features
    Q100_1,V100 = VQlinspace2(batch_combined(i).cycles(100))
    Q10_1,V10 = VQlinspace2(batch_combined(i).cycles(10))
    QDiff = Q100_1 - Q10_1
    feat[i,1] = log10(np.abs(var(QDiff)))

#ylim([0.95,1.1])
plt.xlabel('Q_{100} - Q_{10}')
plt.ylabel('Voltage')
set(gca,'fontsize',16)
##
colormap('jet')
CM = colormap('jet')
plt.figure()
for i in np.arange(1,feat.shape[2-1]+1).reshape(-1):
    for j in np.arange(1,numBat+1).reshape(-1):
        subplot(4,6,i)
        color_ind = np.ceil((bat_label(j) - min_Q) / (max_Q - min_Q) * 64)
        plt.plot(feat(j,i),bat_label(j),'.','Color',CM(color_ind,:),'MarkerSize',15)